In [2]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

# Setup

In [6]:
import pandas as pd

from src.dataset import *
from src.train import *
from src.utils import *

In [7]:
RES_DIR_PATH = "res"
AUDIOS_DIR_PATH = os.path.join(RES_DIR_PATH, "mp3_data")
MODELS_DIR_PATH = os.path.join(RES_DIR_PATH, "models")
DATASETS_DIR_PATH = os.path.join(RES_DIR_PATH, "datasets")

CSV_PATH = os.path.join(RES_DIR_PATH, "samples.csv")

FEATURES_CONFIG = {
    "genre": {"top_n": 3, "samples": 1000}
}

VALID_SIZE = 0.1
TEST_SIZE = 0.1

## Backbones

## Fine-tuning

Thanks to the HuggingFace library, it would be enough to

## Classifier

## Multi-task

# Training

In [8]:
TRAINING_CONFIG = {
    "epochs": 20,
    "learning_rate": 5e-5,
    "warmup": 0.0,
    "train_batch_size": 16,
    "eval_batch_size": 32,
    "feature_encoder": None,
    "freeze_encoder": None,
    "classifier_layers": None, 
    "classifier_dropout": None,
}

In [9]:
# filtered_csv_path = get_csv_name(FEATURES_CONFIG, CSV_PATH)
# encoder_ds = {}

# for encoder in ["wav2vec2", "whisper"]:
#     encoded_dataset_path = os.path.join(DATASETS_DIR_PATH, f"ds-{encoder}-full-encoded")

#     df = pd.read_csv(filtered_csv_path)
#     ds = datasets.load_from_disk(encoded_dataset_path)
#     ds = add_audio_column(ds, audios_dir_path=AUDIOS_DIR_PATH, training_config={"feature_encoder": encoder})
#     encoder_ds[encoder] = prepare_ds(ds, df, FEATURES_CONFIG, 0.2, fixed_mapping=None, save=False)

## Baseline

In [10]:
TRAINING_CONFIG["feature_encoder"] = "wav2vec2"
TRAINING_CONFIG["freeze_encoder"] = True
TRAINING_CONFIG["classifier_layers"] = [256]
TRAINING_CONFIG["classifier_dropout"] = 0

In [11]:
filtered_csv_path = get_csv_name(FEATURES_CONFIG, CSV_PATH)
encoded_dataset_path = os.path.join(DATASETS_DIR_PATH, f"ds-{TRAINING_CONFIG['feature_encoder']}-full-encoded")


filtered_csv_path = get_csv_name(FEATURES_CONFIG, CSV_PATH)

if os.path.exists(filtered_csv_path):
    print(f"Loading {filtered_csv_path}")
    df = pd.read_csv(filtered_csv_path)
else:
    df = pd.read_csv(CSV_PATH)
    df = filter_df(
        df, 
        remove_nones=False,
        features_config=FEATURES_CONFIG, 
    )
    df.to_csv(filtered_csv_path, index=False)

print(f"{len(df)} examples in DataFrame")

ds = datasets.load_from_disk(encoded_dataset_path)
ds = add_audio_column(ds, audios_dir_path=AUDIOS_DIR_PATH, training_config={"feature_encoder": TRAINING_CONFIG['feature_encoder']})
ds = prepare_ds(ds, df, FEATURES_CONFIG, 0.2, fixed_mapping=None, save=False)
ds = ds.rename_column("label_genre", "label")

# ds = encoder_ds[TRAINING_CONFIG["feature_encoder"]]

Loading /content/drive/MyDrive/Projects/music-classification/res/samples_genre3s1000.csv
999 examples in DataFrame


/usr/local/lib/python3.8/dist-packages/transformers/configuration_utils.py:375: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


Casting the dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

In [12]:
TRAINING_CONFIG["warmup"] = 0.0
TRAINING_CONFIG["freeze_encoder"] = False

run_name = "baseline"
model = get_model(TRAINING_CONFIG, ds["train"])

trainer = get_trainer(
    run_name=run_name,
    model=model,
    train_ds=ds["train"],
    eval_ds=ds["test"],
    training_config=TRAINING_CONFIG,
    feature_extractor=None,
    output_dir="out",
    debug=True,
    env=NOTEBOOK_ENV,
)

trainer.train()
end_training(run_name, trainer, MODELS_DIR_PATH)

Some weights of the model checkpoint at facebook/wav2vec2-base were not used when initializing Wav2Vec2ForSequenceClassification: ['project_q.weight', 'quantizer.weight_proj.bias', 'project_q.bias', 'quantizer.weight_proj.weight', 'quantizer.codevectors', 'project_hid.bias', 'project_hid.weight']
- This IS expected if you are initializing Wav2Vec2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['projector.weight', 'classifier.bias', 'projector.

The following columns in the training set don't have a corresponding argument in `Wav2Vec2ForSequenceClassification.forward` and have been ignored: id, duration. If id, duration are not expected by `Wav2Vec2ForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 608
  Num Epochs = 20
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 760
  Number of trainable parameters = 94569347
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Epoch,Training Loss,Validation Loss,Accuracy
1,1.046600,1.020314,0.549020
2,0.975800,0.917156,0.614379
3,0.911300,0.848616,0.614379
4,0.825900,0.842709,0.620915
5,0.705200,0.862440,0.653595
6,0.663200,0.781739,0.705882
7,0.511500,0.701397,0.732026
8,0.476700,0.804047,0.725490
9,0.396500,0.931263,0.673203
10,0.332300,0.993699,0.692810


The following columns in the evaluation set don't have a corresponding argument in `Wav2Vec2ForSequenceClassification.forward` and have been ignored: id, duration. If id, duration are not expected by `Wav2Vec2ForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 153
  Batch size = 32
Saving model checkpoint to out/checkpoint-38
Configuration saved in out/checkpoint-38/config.json
Model weights saved in out/checkpoint-38/pytorch_model.bin
Feature extractor saved in out/checkpoint-38/preprocessor_config.json
Deleting older checkpoint [out/checkpoint-418] due to args.save_total_limit
The following columns in the evaluation set don't have a corresponding argument in `Wav2Vec2ForSequenceClassification.forward` and have been ignored: id, duration. If id, duration are not expected by `Wav2Vec2ForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 153
  Batch siz

eval/accuracy,▁▃▃▃▄▆▇▇▅▆▆▇▇▇▇▇██▇█
eval/loss,▅▄▃▃▃▂▁▂▄▄▄▅▅▇██▇██▇
eval/runtime,▄▁▅▂▄█▁▃▃▃▂▃▂▃▃█▄▅▂▃
eval/samples_per_second,▅█▄▇▅▁█▆▆▆▇▅▇▆▆▁▅▄▇▆
eval/steps_per_second,▅█▄▇▅▁█▆▆▆▇▅▇▆▆▁▅▄▇▆
train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇████
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇████
train/learning_rate,██▇▇▇▆▆▆▅▅▅▄▄▃▃▃▂▂▂▁▁
train/loss,██▇▇▆▅▅▄▄▃▃▃▃▂▂▁▁▁▁▁▁
train/total_flos,▁
train/train_loss,▁


Saving model checkpoint to /content/drive/MyDrive/Projects/music-classification/res/models/baseline
Configuration saved in /content/drive/MyDrive/Projects/music-classification/res/models/baseline/config.json
Model weights saved in /content/drive/MyDrive/Projects/music-classification/res/models/baseline/pytorch_model.bin
Feature extractor saved in /content/drive/MyDrive/Projects/music-classification/res/models/baseline/preprocessor_config.json


## Wav2Vec2

In [14]:
permutations = {
    "freeze_encoder": [True, False],
    "classifier_layers": [[256], [256, 256]],
}

In [15]:
for config in permutation:
    ds = datasets[config["feature_encoder"]]
    run_name = get_run_name(config)
    model = get_model(config, ds)

    trainer = get_trainer(
        run_name=run_name,
        model=model,
        train_ds=ds["train"],
        eval_ds=ds["valid"],
        training_config=config,
        feature_extractor=None,
        output_dir="out",
        debug=True,
        env=NOTEBOOK_ENV,
    )

    trainer.train()
    end_training(run_name, trainer, MODELS_DIR_PATH)

NameError: ignored

## Whisper